In [42]:
# Standard libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Handeling .wav files

import librosa
from librosa import feature

# Machine Learning

from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold

from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from functools import reduce

# data vizualisation

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
def merge_pd(machine):
    
    # Opening the .csv files
    
    df_6dB=pd.read_csv(f'Librosa_features_{machine}_6dB.csv')
    df_0dB=pd.read_csv(f'Librosa_features_{machine}_0dB.csv')
    df_min6dB=pd.read_csv(f'Librosa_features_{machine}_-6dB.csv')
    
    # Dropping the 'Unnamed: 0' column
    
    df_6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_0dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_min6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    

    
    # Merging the .csv files into one DataFrame
    
    data_frames = [df_6dB, df_0dB, df_min6dB]
    df_merged = pd.concat(data_frames)
    
    
    return df_merged

In [44]:
fan = merge_pd('fan')

In [45]:
minority_class_len = len(fan[fan['normal(0)/abnormal(1)'] == 1])
majority_class_indices = fan[fan['normal(0)/abnormal(1)'] == 0].index
random_majority_indices = np.random.choice(majority_class_indices, minority_class_len , replace= False)
minority_class_indices  = fan[fan['normal(0)/abnormal(1)'] == 1].index

under_sample_indices = np.concatenate( [minority_class_indices , random_majority_indices])
under_sample = fan.loc[under_sample_indices]
X = under_sample.drop(columns = ['normal(0)/abnormal(1)'])
y = under_sample['normal(0)/abnormal(1)']

# 20% of the overal data will seperated for later validation of the model

X_model, X_valid, y_model, y_valid = train_test_split(X, y, test_size=0.2, random_state = 42, stratify = y)

# 60% of the overal data (75% of X_model, y_model) will be used to create a training set for the model
# 20% of the overal data (25% of X_model, y_model) will be used to create a testing set for the model

X_train, X_test, y_train, y_test = train_test_split(X_model,
                                                   y_model,
                                                   test_size = 0.25,
                                                   random_state = 10, stratify=y_model)

In [46]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

In [47]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 56em; }</style>"))

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.feature_selection import RFE

y_pred = []
clfs = []

clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())
clfs.append(RandomForestClassifier(class_weight='balanced'))
#clfs.append(EasyEnsembleClassifier(importance_getter=index))

#feature_list = [6,13,21]

#for index in feature_list:
for index in range(4,22):
    
    for classifier in clfs:
        
        print("----------------------------------------------")
        print("----------------------------------------------")
        print(classifier)
        print("----------------------------------------------")
        print("----------------------------------------------")
        
        sel = RFE(classifier, n_features_to_select = index)
        sel.fit(X_train, y_train)
        features = X_train.columns[sel.get_support()]
        X_train_rfe = sel.transform(X_train)
        X_test_rfe = sel.transform(X_test)
        print('Selected Feature', index)
        print(features)
    
        classifier.fit(X_train_rfe, y_train)
        y_pred= classifier.predict(X_test_rfe)
        scores = cross_val_score(pipeline, X_train_rfe, y_train, cv=5)


        print("----------------------------------------------")
        print("TRAIN-TEST")
        print("----------------------------------------------")


        print('confusion matrix', classifier)
        print(confusion_matrix(y_test, y_pred))
        print('classification report')
        print(classification_report(y_test, y_pred))
        print('accuracy score')
        print(accuracy_score(y_test, y_pred))
        
        X_valid_rfe = sel.transform(X_valid)
        y_pred = classifier.predict(X_valid_rfe)

        print("----------------------------------------------")
        print("TRAIN-VALIDATION")
        print("----------------------------------------------")

        print('confusion matrix', classifier)
        print(confusion_matrix(y_valid, y_pred))
        print('classification report')
        print(classification_report(y_valid, y_pred))
        print('accuracy score')
        print(accuracy_score(y_valid, y_pred))

----------------------------------------------
----------------------------------------------
DecisionTreeClassifier()
----------------------------------------------
----------------------------------------------
Selected Feature 4
Index(['melspectrogram_sum', 'mfcc', 'rms', 'spectral_flatness '], dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2470  185]
 [  99 2556]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.93      0.95      2655
           1       0.93      0.96      0.95      2655

    accuracy                           0.95      5310
   macro avg       0.95      0.95      0.95      5310
weighted avg       0.95      0.95      0.95      5310

accuracy score
0.9465160075329567
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix 

Selected Feature 6
Index(['melspectrogram', 'melspectrogram_sum', 'melspectrogram_std', 'mfcc',
       'rms', 'spectral_flatness '],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2462  193]
 [  84 2571]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.93      0.95      2655
           1       0.93      0.97      0.95      2655

    accuracy                           0.95      5310
   macro avg       0.95      0.95      0.95      5310
weighted avg       0.95      0.95      0.95      5310

accuracy score
0.947834274952919
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2478  177]
 [  87 2568]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.9

Selected Feature 8
Index(['melspectrogram', 'melspectrogram_sum', 'melspectrogram_std', 'mfcc',
       'rms', 'spectral_centroid', 'spectral_contrast', 'spectral_flatness '],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2490  165]
 [  72 2583]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.94      0.95      2655
           1       0.94      0.97      0.96      2655

    accuracy                           0.96      5310
   macro avg       0.96      0.96      0.96      5310
weighted avg       0.96      0.96      0.96      5310

accuracy score
0.9553672316384181
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2488  167]
 [  75 2580]]
classification report
              precision    recall  f1-score 

Selected Feature 10
Index(['melspectrogram', 'melspectrogram_sum', 'melspectrogram_std', 'mfcc',
       'rms', 'spectral_bandwidth', 'spectral_contrast', 'spectral_flatness ',
       'spectral_rolloff', 'zero_crossing_rate'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2473  182]
 [  98 2557]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.93      0.95      2655
           1       0.93      0.96      0.95      2655

    accuracy                           0.95      5310
   macro avg       0.95      0.95      0.95      5310
weighted avg       0.95      0.95      0.95      5310

accuracy score
0.9472693032015066
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2493  162]
 [  88 2567]]
classification 

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[2576   79]
 [  68 2587]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2655
           1       0.97      0.97      0.97      2655

    accuracy                           0.97      5310
   macro avg       0.97      0.97      0.97      5310
weighted avg       0.97      0.97      0.97      5310

accuracy score
0.9723163841807909
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[2575   80]
 [  64 2591]]
classification report
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2655
           1       0.97      0.98      0.97      2655

    accuracy                   

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[2580   75]
 [  66 2589]]
classification report
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2655
           1       0.97      0.98      0.97      2655

    accuracy                           0.97      5310
   macro avg       0.97      0.97      0.97      5310
weighted avg       0.97      0.97      0.97      5310

accuracy score
0.9734463276836158
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier()
[[2582   73]
 [  63 2592]]
classification report
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2655
           1       0.97      0.98      0.97      2655

    accuracy                           0.97      5310
   macro avg       0.97

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2484  171]
 [  80 2575]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.94      0.95      2655
           1       0.94      0.97      0.95      2655

    accuracy                           0.95      5310
   macro avg       0.95      0.95      0.95      5310
weighted avg       0.95      0.95      0.95      5310

accuracy score
0.9527306967984934
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2495  160]
 [  73 2582]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.94      0.96      2655
           1       0.94      0.97      0.96      2655

    accuracy                           0.96      5310
   macro avg       0.96

----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[2572   83]
 [  70 2585]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2655
           1       0.97      0.97      0.97      2655

    accuracy                           0.97      5310
   macro avg       0.97      0.97      0.97      5310
weighted avg       0.97      0.97      0.97      5310

accuracy score
0.9711864406779661
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix RandomForestClassifier(class_weight='balanced')
[[2578   77]
 [  65 2590]]
classification report
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2655
           1       0.97      0.98      0.97      2655

    accuracy                   

Selected Feature 18
Index(['melspectrogram', 'melspectrogram_max', 'melspectrogram_sum',
       'melspectrogram_corr', 'melspectrogram_std', 'mfcc', 'rms',
       'spectral_centroid', 'spectral_bandwidth', 'spectral_contrast',
       'spectral_flatness ', 'spectral_rolloff', 'zero_crossing_rate',
       'mean perc', 'max harm', 'max perc', 'min harm', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix RandomForestClassifier()
[[2582   73]
 [  80 2575]]
classification report
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2655
           1       0.97      0.97      0.97      2655

    accuracy                           0.97      5310
   macro avg       0.97      0.97      0.97      5310
weighted avg       0.97      0.97      0.97      5310

accuracy score
0.9711864406779661
----------------------------------------------
TRAIN-V

Selected Feature 20
Index(['melspectrogram', 'melspectrogram_min', 'melspectrogram_max',
       'melspectrogram_sum', 'melspectrogram_corr', 'melspectrogram_std',
       'mfcc', 'rms', 'spectral_centroid', 'spectral_bandwidth',
       'spectral_contrast', 'spectral_flatness ', 'spectral_rolloff',
       'zero_crossing_rate', 'mean harm', 'mean perc', 'max harm', 'max perc',
       'min harm', 'min perc'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2460  195]
 [  92 2563]]
classification report
              precision    recall  f1-score   support

           0       0.96      0.93      0.94      2655
           1       0.93      0.97      0.95      2655

    accuracy                           0.95      5310
   macro avg       0.95      0.95      0.95      5310
weighted avg       0.95      0.95      0.95      5310

accuracy score
0.9459510357815443
------------